# Neighborhood Project

## CMPINF0999 Final

### Yifeng Qiu and James Fenn

In [1]:
# load libraries
import pandas as pd
%matplotlib inline

In [ ]:
# basic arrest / "people" data; may depend on population of each neighborhood, unreliable unless scaled accordingly

# download 311 data from https://data.wprdc.org/dataset/311-data
data311 = pd.read_csv("https://data.wprdc.org/datastore/dump/76fda9d0-69be-4dd5-8108-0de7907fc5a4", low_memory=False)
# download NTC data from https://data.wprdc.org/dataset/non-traffic-citations
dataNTC = pd.read_csv("https://data.wprdc.org/datastore/dump/6b11e87d-1216-463d-bbd3-37460e539d86", low_memory=False)
# download Police Blotter data from https://data.wprdc.org/dataset/uniform-crime-reporting-data
dataBlotter = pd.read_csv("https://data.wprdc.org/datastore/dump/044f2016-1dfd-4ab0-bc1e-065da05fca2e", low_memory=False)
# download arrest data from https://data.wprdc.org/dataset/arrest-data
dataArrests = pd.read_csv("https://data.wprdc.org/datastore/dump/e03a89dd-134a-4ee8-a2bd-62c40aeebc6f", low_memory=False)

In [4]:
# information about the environment / physical aspects of each neighborhood
# how many playgrounds are there, how often do things catch fire, are their many trees, what is the air quality, etc

# download fire incidents from https://data.wprdc.org/dataset/fire-incidents-in-city-of-pittsburgh
dataFires = pd.read_csv("https://data.wprdc.org/datastore/dump/8d76ac6b-5ae8-4428-82a4-043130d17b02", low_memory=False)
# download trees data from https://data.wprdc.org/dataset/city-trees
dataTrees = pd.read_csv("https://data.wprdc.org/datastore/dump/1515a93c-73e3-4425-9b35-1cd11b2196da", low_memory=False)
# download public playgrounds from https://data.wprdc.org/dataset/playgrounds
dataPlaygrounds = pd.read_csv("https://data.wprdc.org/datastore/dump/47350364-44a8-4d15-b6e0-5f79ddff9367", low_memory=False)
# download athletics; courts and rinks data from https://data.wprdc.org/dataset/city-of-pittsburgh-courts
dataCourtsRinks = pd.read_csv("https://data.wprdc.org/datastore/dump/a5b71bfa-840c-4c86-8f43-07a9ae854227", low_memory=False)
# download athetics; playing fields data from https://data.wprdc.org/dataset/playing-fields
dataPlayingFields = pd.read_csv("https://data.wprdc.org/datastore/dump/6af89346-b971-41d5-af09-49cfdb4dfe23", low_memory=False)

In [ ]:
# "things that might affect people" info; are there obstructions to the right of way,
# what ISPs do people use, electric companies, etc?

# download right of way data from https://data.wprdc.org/dataset/right-of-way-permits
dataROW = pd.read_csv("https://data.wprdc.org/datastore/dump/cc17ee69-b4c8-4b0c-8059-23af341c9214", low_memory=False)
# download ISP data (2016)
dataISP = pd.read_csv("https://data.wprdc.org/dataset/ae1f7cda-5e15-4a8a-a5b2-2e4803f1500a/resource/c2df1e6f-5563-4e53-9de8-b0e4c7d2cb93/download/pittsburghispsbyblock.csv", low_memory=False)
# download energy use data (2009-2014) from https://data.wprdc.org/dataset/municipal-building-energy-usage
dataEnergy = pd.read_csv("https://data.wprdc.org/dataset/7b8896c5-867d-428d-a2e3-bdb732655846/resource/90e87a5c-0cdd-4534-ab84-0cc523b110f5/download/municpal-building-energy-use-2009-2014.csv", low_memory=False)